# Homework I

Joana Peixinho (ist1103335) & Miguel Fernandes (ist1103573)

## I. Pen and Paper [11v]



### 1) [6v] Perform one epoch of the EM clustering algorithm and determine the new parameters. 
Hint: we suggest you to use numpy and scipy, however disclose the intermediary results step by step

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

observations = np.array([
    [1, 0.6, 0.1],
    [0, -0.4, 0.8],
    [0, 0.2, 0.5],
    [1, 0.4, -0.1]
])


K = 2


pi = np.array([0.5, 0.5])
p = np.array([0.3, 0.7])
mu = np.array([[1, 1], [0, 0]])
cov = np.array([[[2, 0.5], [0.5, 2]], [[1.5, 1], [1, 1.5]]])

# EM Clustering
num_iterations = 1

for _ in range(num_iterations):
    # Expectation step
    cluster_probabilities = np.zeros((len(observations), K))
    for k in range(K):
        cluster_probabilities[:, k] = pi[k] * multivariate_normal.pdf(observations[:, 1:], mean=mu[k], cov=cov[k])
    
    cluster_probabilities /= cluster_probabilities.sum(axis=1)[:, np.newaxis]

    # Maximization step
    Nk = cluster_probabilities.sum(axis=0)
    pi = Nk / len(observations)
    p = cluster_probabilities.T.dot(observations[:, 0]) / Nk
    for k in range(K):
        x_minus_mu = observations[:, 1:] - p[k]
        cov[k] = np.dot(x_minus_mu.T, (x_minus_mu * cluster_probabilities[:, k][:, np.newaxis])) / Nk[k]


print("Probabilidades dos Clusters:")
print(pi)
print("Probabilidade p1 e p2:")
print(p)
print("Matrizes de Covariância:")
print(cov)


### 2)  [2v] Given the new observation, 𝐱𝑛𝑒𝑤 , determine the cluster memberships (posteriors).

### 3) [2.5v] Performing a hard assignment of observations to clusters under a ML assumption, identify the silhouette of the larger cluster under a Manhattan distance

### 4) [0.5v] Knowing the purity of the clustering solution is 0.75, identify the number of possible classes (ground truth).


## Programming [9v]

Recall the `column_diagnosis.arff` dataset from previous homeworks. For the following exercises,
normalize the data using sklearn’s MinMaxScaler 

In [ ]:
#Load the data

import pandas as pd
import numpy as np
from scipy.io.arff import loadarff

# Reading the ARFF file
data = loadarff('column_diagnosis.arff')
df = pd.DataFrame(data[0])
df['class'] = df['class'].str.decode('utf-8')


### 1) 
[4v] Using sklearn, apply k-means clustering fully unsupervisedly on the normalized data with
𝑘 ∈ {2,3,4,5} (random=0 and remaining parameters as default). Assess the silhouette and purity of
the produced solutions.


In [ ]:
from sklearn import datasets, metrics, cluster, mixture
from sklearn.preprocessing import MinMaxScaler


k = [2, 3, 4, 5]

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

#normalize the data
X, y = df.drop('class', axis=1), df['class']
X = MinMaxScaler().fit_transform(X)

# K-Means for different values of K and print the results

for i in k:
    kmeans = cluster.KMeans(n_clusters=i, random_state=0).fit(X)
    print("K-Means with k = " + str(i))
    print("Silhouette Score: " + str(metrics.silhouette_score(X, kmeans.labels_)))
    print("Purity Score: " + str(purity_score(y, kmeans.labels_)))
    print("\n")


### 2)
[2v] Consider the application of PCA after the data normalization:
i. Identify the variability explained by the top two principal components.
ii. For each one of these two components, sort the input variables by relevance by
inspecting the absolute weights of the linear projection

### 3) 
[2v] Visualize side-by-side the data using: i) the ground diagnoses, and ii) the previously learned
𝑘 = 3 clustering solution. To this end, projected the normalized data onto a 2-dimensional data
space using PCA and then color observations using the reference and cluster annotations

### 4)
[1v] Considering the results from questions (1) and (3), identify two ways on how clustering can
be used to characterize the population of ill and healthy individuals

